Uma empresa de Contact Center e Customer Experience contrata muitas pessoas por ano. Esse ramo de Contact Center é conhecido por ser a porta de entrada de muita gente no mercado, pois não exige muita experiência.

No entanto, o turnover por baixo desempenho dos colaboradores também é muito alto, fazendo com que a empresa tenha que ficar contratando e treinando pessoas constantemente para que a operação de atendimento não fique sem colaboradores e o cliente final não sinta uma demora para ser atendimento nos mais diversos canais de atendimento, desde o telefone até o chat e whatsapp.

A empresa gostaria de testar técnicas de People Analytics para ajudar o time de Atração de Talentos a filtrar melhor os perfis de colaboradores no processo seletivo a fim de trazer mais colaboradores com potencial de boa performance no trabalho. Para isso, os analistas de dados devem analisar um histórico de 7.240 colaboradores e sua performance após 6 meses a contração para responder:

 
Com base nessas informações, pede-se:
1.     Quais fatores o time de Atração de Talentos deve olhar para selecionar candidatos com maior chance de ter boa performance após 6 meses à data da contratação?
2.     Com base nisso, qual política de contratação deve passar a valer de forma que a empresa aumente as boas contratações?



In [2]:
#Manipulação dos dados 
import pandas as pd
import numpy as np

#Pacotes gráficos
import matplotlib.pyplot as plt
import seaborn as sns

#Analises
from sklearn import linear_model
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score, average_precision_score

# Módulo de Análise Exploratória
#import aed


In [3]:
#Leitura da base

dados=pd.read_excel("base_atendimento.xlsx", sheet_name=1)



### Ajuste do banco de dados

In [4]:

#dados.dtypes
dados.columns



Index(['Id_Agente', 'Boa_Performance',
       'P 01: Antes de trabalhar na empresa, você tinha experiência em Contact Center?',
       'P 02: Qual área de atendimento você atua ou atuou por mais tempo?',
       'P 03: Qual a sua escolaridade?',
       'P 04: Qual seu conhecimento em inglês?',
       'P 05: Qual o tempo gasto no trajeto da sua residência até a empresa?',
       'P 06: Você possui dependentes?',
       'P 07: É bom trabalhar com meus colegas de trabalho.',
       'P 08: Não tenho medo de criticar pessoas em cargos de liderança.',
       'P 09: Fico tenso quando estou trabalhando.',
       'P 10: Não gosto de situações que requerem que eu me comporte de forma controlada.',
       'P 11: Gosto de dizer o que penso.',
       'P 12: Gosto de fazer as coisas do meu jeito e não ligo para o que possam pensar.'],
      dtype='object')

In [3]:
#Padronização dos valores '3 - Indiferente' para 'Indiferente'

dados = dados.replace(r'\d+ - ', ' ', regex=True)



#dados = dados.replace(r'[a-zA-Z]', ' ', regex=True) se quiser mudar texto
#dados['bom'] = dados['bom'].str.replace(r'\d+ - ', ' ') se quiser somente na variavel
#dados['texto'] = dados['texto'].str.replace(r'[a-zA-Z]+', ' ')  se quiser somente na variavel

In [5]:
#Retirada dos ' Sem resposta' (existem dois tipos: os que possuem um espaço antes e os que possuem dois)
#Rodar separado. Junto esta dando erro


dados.replace(' Sem resposta', np.nan, inplace=True)
#dados.replace('  Sem resposta', np.nan, inplace=True)
dados = dados.dropna()



#dados = dados.applymap(lambda x: np.nan if x == ' Sem resposta' or x == '  Sem resposta' else x) (outra maneira)

In [139]:
dados 

,Id_Agente,Boa_Performance,P 01,P 02,P 06,P 07,P 08,P 09,P 10,P 11,P 12,P 03_enc,P 04_enc,P 05_enc
5,6,1,Sim,Atendimento Receptivo,Não,Concordo totalmente,Discordo totalmente,Indiferente,Discordo totalmente,Concordo,Discordo totalmente,1,2,3
6,7,0,Sim,Receptivo de Vendas,Sim,Indiferente,Indiferente,Indiferente,Indiferente,Indiferente,Indiferente,1,1,5
23,24,1,Sim,Retenção,Sim,Concordo totalmente,Discordo,Discordo,Indiferente,Concordo,Discordo,3,2,3
33,34,0,Sim,Atendimento Receptivo,Não,Concordo totalmente,Discordo,Indiferente,Indiferente,Concordo,Concordo,1,1,3
36,37,1,Sim,Multiskill,Não,Concordo totalmente,Concordo,Concordo totalmente,Discordo totalmente,Discordo totalmente,Discordo totalmente,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,7226,1,Sim,Atendimento Receptivo,Sim,Concordo,Discordo,Discordo,Indiferente,Discordo,Discordo,1,2,5
7226,7227,1,Sim,Atendimento Receptivo,Não,Concordo totalmente,Discordo totalmente,Indiferente,Indiferente,Concordo,Discordo totalmente,4,2,4
7236,7237,1,Sim,Atendimento Receptivo,Sim,Concordo totalmente,Discordo,Discordo,Discordo,Concordo,Discordo,1,1,1
7238,7239,1,Sim,Backoffice,Não,Concordo totalmente,Discordo,Discordo,Discordo,Discordo,Discordo,1,3,3


In [6]:
#Divisão BD

y=dados['Boa_Performance']



In [7]:
#Renome das variaveis

dados=dados.rename(columns={'P 01: Antes de trabalhar na empresa, você tinha experiência em Contact Center?':'P 01',
       'P 02: Qual área de atendimento você atua ou atuou por mais tempo?':'P 02',
       'P 03: Qual a sua escolaridade?':'P 03',
       'P 04: Qual seu conhecimento em inglês?':'P 04',
       'P 05: Qual o tempo gasto no trajeto da sua residência até a empresa?':'P 05',
       'P 06: Você possui dependentes?':'P 06',
       'P 07: É bom trabalhar com meus colegas de trabalho.':'P 07',
       'P 08: Não tenho medo de criticar pessoas em cargos de liderança.':'P 08',
       'P 09: Fico tenso quando estou trabalhando.':'P 09',
       'P 10: Não gosto de situações que requerem que eu me comporte de forma controlada.':'P 10',
       'P 11: Gosto de dizer o que penso.':'P 11',
       'P 12: Gosto de fazer as coisas do meu jeito e não ligo para o que possam pensar.':'P 12'})

In [8]:
# Ordenamento das variaveis qualitativas ordinais

P03_map = {" Ensino Médio completo":1," Ensino Superior incompleto":2," Ensino Superior cursando":3,
            " Ensino Superior concluído":4," Pós Graduação cursando":5," Pós Graduação concluído":6}
P04_map = {" Não tenho conhecimento em inglês":1," Básico":2," Intermediário":3," Avançado":4," Fluente":5}
P05_map = {" Até 30min":1," De 30min à 1h":2," De 1h à 1h30":3," De 1h30 à 2h":4," Acima de 2h":5}
        
dados["P 03_enc"] = dados["P 03"].map(P03_map)    
dados["P 04_enc"] = dados["P 04"].map(P04_map)   
dados["P 05_enc"] = dados["P 05"].map(P05_map)   

dados.drop(columns=['P 03', 'P 04', 'P 05'], inplace=True)




### Analise exploratoria 

PELA ANALISE EXPLORATORIA PODEMOS OBSERVAR......................

In [22]:

import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')
sns.pairplot(dados, hue='Boa_Performance');


KeyError: 'Boa_Performance'

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.heatmap(dados.corr(),
            cmap='RdBu_r',
            annot=True);

In [25]:
# Analise IV
dados_iv = aed.analise_iv(dados,'Boa_Performance')
dados_iv.get_lista_iv()

C:\Users\tania\Desktop\Preditiva\Estudos de caso\performance colaboradores\aed.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num[nome_var] = pd.qcut(df_num[var],


Boa_Performance,IV
Variavel,
P 02: Qual área de atendimento você atua ou atuou por mais tempo?,0.03
P 06: Você possui dependentes?,0.03
"P 01: Antes de trabalhar na empresa, você tinha experiência em Contact Center?",0.02
P 07: É bom trabalhar com meus colegas de trabalho.,0.02
P 03: Qual a sua escolaridade?,0.01
P 08: Não tenho medo de criticar pessoas em cargos de liderança.,0.01
Id_Agente,0.00
P 04: Qual seu conhecimento em inglês?,0.00
P 05: Qual o tempo gasto no trajeto da sua residência até a empresa?,0.00


### Criação do modelo Logistico de regressão 

PARA ESSE PROJETO FOI OPTADO PELA CRIACAO DE OM MODELO LOGISTICO, ONDE ALEM DE PODER IDENTIFICAR AS VARIAVEIS MAIS SIGNIFICATIVAS PARA EXPLICAR A PERFORMANCE DO COLABORADOR COMO ATRIBUIR AO COLABORADOR UM SCORE

O MODELO CRIADO FICOU UM POUCO EXTENSO, COM 37 COEFICIENTES. SENDO ESSE UM POSSIVEL MOTIVO PELO QUAIL AS METRICAS DE VERIFICACAO DE QUALIDADE DO MODELO DERAM RELATIVAMENTE BAIXAS. COM UMA ACURACIA DE 57% E UM F1 SCORE DE 62%

REALIZAR UM SUMMARY DO MODELO.FIT A FIM DE VER OS COEFICIENTES E SEUS P VALORES SIGNIFICARIVOS OU NAO

FOI CRIADA UMA PLANILHA EM EXCEL NA PASTA DESSE PROJETO COM OS DADOS DOS SCORES PESSOAL PARA CADA COLABORADOR A FIM DE FAZER FUTURAS ANALISES. OS PRIMEIROS COLOCADOS COM MELHOR PROBABILIDADE DE BOA PERFORMANCE SAO OS NUMEROS "22, 33, 44, 69, 67" 

In [9]:
dados.drop(columns=['Boa_Performance'], inplace=True)
dados = pd.get_dummies(dados, drop_first=True) 
   

In [10]:
pd.set_option('display.max_columns', None)
dados

,Id_Agente,P 03_enc,P 04_enc,P 05_enc,P 02_ Ativo de Vendas,P 02_ Backoffice,P 02_ Chat ou email,P 02_ Cobrança,P 02_ Multiskill,P 02_ Receptivo de Vendas,P 02_ Retenção,P 02_ Suporte Técnico,P 02_ Suporte Técnico com Vendas,P 06_ Sim,P 07_ Concordo totalmente,P 07_ Discordo,P 07_ Discordo totalmente,P 07_ Indiferente,P 08_ Concordo totalmente,P 08_ Discordo,P 08_ Discordo totalmente,P 08_ Indiferente,P 09_ Concordo totalmente,P 09_ Discordo,P 09_ Discordo totalmente,P 09_ Indiferente,P 10_ Concordo totalmente,P 10_ Discordo,P 10_ Discordo totalmente,P 10_ Indiferente,P 11_ Concordo totalmente,P 11_ Discordo,P 11_ Discordo totalmente,P 11_ Indiferente,P 12_ Concordo totalmente,P 12_ Discordo,P 12_ Discordo totalmente,P 12_ Indiferente
5,6,1,2,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
6,7,1,1,5,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
23,24,3,2,3,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
33,34,1,1,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
36,37,1,3,5,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,7226,1,2,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
7226,7227,4,2,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
7236,7237,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
7238,7239,1,3,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0


In [11]:

#Dividir o conjunto de dados em treinamento e teste
dados_train, dados_test, y_train, y_test = train_test_split(dados, y, test_size=0.3, random_state=42)

#Escalar os dados
scaler = StandardScaler()
dados_train_scaled = scaler.fit_transform(dados_train)
dados_test_scaled = scaler.transform(dados_test)

#Criar e treinar o modelo de regressão logística com o solver 'liblinear'
modelo = LogisticRegression(solver='liblinear', random_state=42)
modelo.fit(dados_train_scaled, y_train)

#Realizar previsões no conjunto de teste
y_pred = modelo.predict(dados_test_scaled)


In [13]:
coeficientes = modelo.coef_
print(coeficientes)
print(modelo.fit.summary())


[[-0.06061298  0.17006417 -0.07002519 -0.04581444 -0.10820384  0.04127565
   0.00139255 -0.04428974 -0.0514554   0.00601487 -0.11823568 -0.14824226
  -0.01601874 -0.11441126  0.098159   -0.05245756  0.05993955 -0.22110127
  -0.08237138 -0.10178203 -0.00617928  0.04732008  0.12019031  0.03411886
  -0.0084902  -0.01698808 -0.04553881 -0.32629963 -0.15361151 -0.14366437
  -0.11931113 -0.12656125  0.07878076  0.06550737 -0.08319771 -0.10806825
   0.05391551 -0.12274338]]


In [14]:
# Obtendo os p-valores dos coeficientes
from scipy.stats import norm

# Calculando os p-valores
n = dados_train_scaled.shape[0]
p = dados_train_scaled.shape[1]
p_valores = 2 * (1 - norm.cdf(abs(coeficientes)))  # Utilizando a distribuição normal para cálculo do p-valor

# Imprimindo os coeficientes e seus respectivos p-valores
for i in range(p):
    print(f"Coeficiente {i}: {coeficientes[0][i]}, p-valor: {p_valores[0][i]}")

Coeficiente 0: -0.06061298367964191, p-valor: 0.9516674330567281
Coeficiente 1: 0.17006416547260117, p-valor: 0.8649596747573618
Coeficiente 2: -0.07002518554741606, p-valor: 0.9441736136700709
Coeficiente 3: -0.04581443897400637, p-valor: 0.9634581502583994
Coeficiente 4: -0.10820384007311401, p-valor: 0.9138339989052011
Coeficiente 5: 0.04127564836537418, p-valor: 0.9670761463113056
Coeficiente 6: 0.0013925472493661122, p-valor: 0.9988889084086447
Coeficiente 7: -0.044289744937538074, p-valor: 0.9646734460089967
Coeficiente 8: -0.051455396263901124, p-valor: 0.9589626433625156
Coeficiente 9: 0.006014871615203295, p-valor: 0.9952008557408198
Coeficiente 10: -0.11823568085665456, p-valor: 0.9058809189952683
Coeficiente 11: -0.14824225836480887, p-valor: 0.8821515828704405
Coeficiente 12: -0.016018743559661142, p-valor: 0.98721943841495
Coeficiente 13: -0.11441126053172983, p-valor: 0.9089117879980932
Coeficiente 14: 0.09815899952256649, p-valor: 0.921806038825757
Coeficiente 15: -0.052

#### Medidas de analise do modelo

In [15]:
#Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

# Calcular a precisão do modelo
precision = precision_score(y_test, y_pred)
print("Precisão do modelo:", precision)

# Calcular o recall (sensibilidade) do modelo
recall = recall_score(y_test, y_pred)
print("Recall do modelo:", recall)

# Calcular o F1-score do modelo
f1 = f1_score(y_test, y_pred)
print("F1-score do modelo:", f1)

# Calcular a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusão:")

print(conf_matrix)

auc_roc = roc_auc_score(y_test, y_pred)
print("AUC-ROC:", auc_roc)

# Calcular o AUC-PR
auc_pr = average_precision_score(y_test, y_pred)
print("AUC-PR:", auc_pr)


Acurácia do modelo: 0.57
Precisão do modelo: 0.5891891891891892
Recall do modelo: 0.6728395061728395
F1-score do modelo: 0.6282420749279538
Matriz de confusão:
[[ 62  76]
 [ 53 109]]
AUC-ROC: 0.5610574342458401
AUC-PR: 0.5730964297630965


####  Probabilidades de cada individuo ter boa performance (Score)

In [16]:
probabilidades_teste = modelo.predict_proba(dados_test)

dados_teste_com_prob = dados_test.copy()
dados_teste_com_prob['Prob_Boa_Performance'] = probabilidades_teste[:, 1]  # Probabilidades da classe 1 (Boa Performace)

# Formate as probabilidades para saírem com no máximo 5 casas decimais
dados_teste_com_prob['Prob_Boa_Performance'] = dados_teste_com_prob['Prob_Boa_Performance'].apply(lambda x: format(x, '.5f'))

# Exiba o DataFrame com a nova coluna de probabilidades
print(dados_teste_com_prob)

      Id_Agente  P 03_enc  P 04_enc  P 05_enc  P 02_ Ativo de Vendas  \
4205       4206         1         2         3                      0   
6277       6278         6         2         1                      0   
1802       1803         1         2         4                      0   
2930       2931         2         2         3                      1   
5944       5945         4         2         3                      0   
...         ...       ...       ...       ...                    ...   
3036       3037         2         2         3                      0   
3856       3857         1         2         4                      0   
60           61         1         2         4                      0   
1118       1119         4         2         1                      0   
6371       6372         4         3         4                      0   

      P 02_ Backoffice  P 02_ Chat ou email  P 02_ Cobrança  P 02_ Multiskill  \
4205                 0                    0           

C:\Users\tania\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [18]:
dados_teste_com_prob["Prob_Boa_Performance"]

4205    0.00000
6277    0.00000
1802    0.00000
2930    0.00000
5944    0.00000
         ...   
3036    0.00000
3856    0.00000
60      0.01531
1118    0.00000
6371    0.00000
Name: prob_inadimplencia, Length: 300, dtype: object

In [17]:
# Converta a coluna 'Prob_Boa_Performance' para valores numéricos
dados_teste_com_prob['Prob_Boa_Performance'] = pd.to_numeric(dados_teste_com_prob['Prob_Boa_Performance'])

# Ordene o DataFrame pelas probabilidades de Boa_Performance em ordem decrescente
dados_teste_com_prob = dados_teste_com_prob.sort_values(by='Prob_Boa_Performance', ascending=False)

dados_teste_com_prob.to_csv("dados_teste_com_prob.csv ")

# Exiba o DataFrame ordenado
print(dados_teste_com_prob)

      Id_Agente  P 03_enc  P 04_enc  P 05_enc  P 02_ Ativo de Vendas  \
23           24         3         2         3                      0   
33           34         1         1         3                      0   
44           45         2         2         2                      0   
69           70         3         3         2                      0   
67           68         3         2         3                      0   
...         ...       ...       ...       ...                    ...   
4021       4022         4         2         3                      0   
5241       5242         2         2         3                      0   
6352       6353         2         2         3                      0   
5341       5342         5         3         4                      0   
6371       6372         4         3         4                      0   

      P 02_ Backoffice  P 02_ Chat ou email  P 02_ Cobrança  P 02_ Multiskill  \
23                   0                    0           

#### Explicação sobre a transformação de escala na regressao logistica.

A escala dos dados é uma transformação linear aplicada a todas as variáveis, de modo que elas tenham média zero e desvio padrão igual a um (ou alguma outra escala específica). Isso é feito para evitar problemas de convergência do algoritmo de otimização (solver) e para garantir que as variáveis estejam em uma mesma escala numérica.

Ao escalar os dados, a magnitude dos coeficientes da regressão logística pode mudar, mas a direção e a relação entre as variáveis explicativas e a variável de resposta permanecem as mesmas.

A interpretação dos coeficientes é baseada na relação logarítmica entre a probabilidade de pertencer à classe positiva (evento de interesse) e o valor das variáveis explicativas. O aumento ou diminuição de uma unidade no valor da variável explicativa afeta o log-odds (logaritmo da razão de chances) da classe positiva.

Por exemplo, suponha que você tem uma variável explicativa X1 e o coeficiente correspondente é 0.5. Isso significa que um aumento de uma unidade em X1 está associado a um aumento de 0.5 no log-odds da classe positiva. Com os dados escalados, os valores numéricos do coeficiente podem ser diferentes, mas a interpretação permanece a mesma: um aumento de uma unidade na variável explicativa está associado a um aumento (ou diminuição) no log-odds da classe positiva.